### Développement d’un pipeline de calcul du TMB

#### Projet 4BiM, 2021

#### Auteurs : Marie Casimir, Loup Petitjean et Nicolas Mendiboure
#### Encadrantes Innate-Pharma : Sabrina Carpentier et Luciana Bastista
#### Encadrante INSA : Maïwenn Pineau

*Pour installer cyvcf2 : pip install cyvcf2*

In [1]:
import cyvcf2
import pandas as pd

In [4]:
## Importation d'un VCF avec la librairie cyvcf2 :

#Général :

CHROM = []
POS = []
REF = []
ALT = []
QUAL = []
FILTER = []

# Détails de la section INFO du VCF :

AN = [] #Total number of alleles in called genotypes
AC = [] #Allele count in genotypes, for each ALT allele, in the same order as listed
AF = [] #Allele Frequency in primary data, for each ALT allele, in the same order as listed
BQ = [] #RMS base quality
SB = [] #Strand bias
FA = [] #Overall fraction of reads supporting ALT
MC = [] #Modification base changes at this position
MT = [] #Modification types at this position
NS = [] #Number of Samples With Data
DP = [] #Total Depth across samples
VT = [] #Variant type, can be SNP, INS or DEL
SS = [] #Variant status relative to non-adjacent Normal,0=wildtype,1=germline,2=somatic,3=LOH,4=post-transcriptional modification,5=unknown
ORIGIN = [] #Where the call originated from, the tumor DNA, RNA, or both
SOMATIC = [] #Indicates if record is a somatic mutation
INDEL = [] #Number of indels for all samples
START = [] #Number of reads starting at this position across all samples
STOP = [] #Number of reads stopping at this position across all samples



for record in cyvcf2.VCF("./test_radia.vcf"):
    CHROM.append(record.CHROM)
    POS.append(record.POS)
    REF.append(record.REF)
    ALT.append(record.ALT)
    QUAL.append(record.QUAL)
    FILTER.append(record.FILTER)
    
    #record.INFO est un objet de type cyvcf, pour extraire les données il faut utiliser .get()
    AN.append(record.INFO.get("AN"))
    AC.append(record.INFO.get("AC"))
    AF.append(record.INFO.get("AF"))
    BQ.append(record.INFO.get("BQ"))
    SB.append(record.INFO.get("SB"))
    FA.append(record.INFO.get("FA"))
    MC.append(record.INFO.get("MC"))
    MT.append(record.INFO.get("MT"))
    NS.append(record.INFO.get("NS"))
    DP.append(record.INFO.get("DP"))
    VT.append(record.INFO.get("VT"))
    SS.append(record.INFO.get("SS"))
    ORIGIN.append(record.INFO.get("ORIGIN"))
    SOMATIC.append(record.INFO.get("SOMATIC"))
    INDEL.append(record.INFO.get("INDEL"))
    START.append(record.INFO.get("START"))
    STOP.append(record.INFO.get("STOP"))
    
df_VCF = pd.DataFrame(list(zip(CHROM, POS, REF, ALT, QUAL, FILTER)), 
                  columns=["CHROM", "POS", "REF", "ALT", "QUAL", "FILTER"])

df_VCF[ ["AN", "AC", "AF", "BQ", "SB", "FA", "MC", 
     "MT", "NS", "DP", "VT", "SS", "ORIGIN", 
     "SOMATIC", "INDEL", "START", "STOP"]] = pd.DataFrame(list(zip (AN, AC, AF, BQ, SB, FA, MC,
                                                                    MT, NS, DP, VT, SS, ORIGIN, 
                                                                    SOMATIC, INDEL, START, STOP)))
df_VCF.head(10)

,CHROM,POS,REF,ALT,QUAL,FILTER,AN,AC,AF,BQ,...,MT,NS,DP,VT,SS,ORIGIN,SOMATIC,INDEL,START,STOP
0,1,29651735,C,[T],0.0,None,2,40,0.1,36,...,SOM,3,413,SNP,2,"DNA,RNA",True,0,5,3
1,1,144852566,G,[A],0.0,None,2,4,0.08,37,...,SOM,3,51,SNP,2,DNA,True,0,0,0
2,1,144951880,A,[T],0.0,None,2,6,0.15,33,...,SOM,2,40,SNP,2,DNA,True,0,2,1
3,1,148342496,C,[G],0.0,None,2,82,0.07,37,...,SOM,3,1187,SNP,2,DNA,True,0,7,7
4,1,149340855,T,[C],0.0,None,2,4,0.1,32,...,SOM,2,42,SNP,2,DNA,True,0,0,2
5,1,155294949,G,"[A, C, T]",0.0,None,4,"(37, 1, 1)","(0.09000000357627869, 0.0, 0.0)",33,...,SOM,3,409,SNP,2,"DNA,RNA",True,0,2,11
6,1,205038731,C,[T],0.0,None,2,8,0.28,35,...,SOM,2,29,SNP,2,DNA,True,0,0,0
7,2,91888009,C,[T],0.0,None,2,5,0.1,37,...,SOM,2,48,SNP,2,DNA,True,0,1,0
8,2,100916306,A,"[G, C]",0.0,None,3,"(125, 10)","(0.9300000071525574, 0.07000000029802322)",34,...,SOM,2,135,SNP,2,DNA,True,0,2,3
9,2,111118911,C,[T],0.0,None,2,6,0.06,36,...,SOM,3,96,SNP,2,DNA,True,0,0,2


In [5]:
mutations = df_VCF['ALT'].values
print(mutations[0:6])

[list(['T']) list(['A']) list(['T']) list(['G']) list(['C'])
 list(['A', 'C', 'T'])]


In [6]:
basic_TMB = len(mutations)
print(basic_TMB)

87


In [8]:
import vcf
def read(file):
    reader = vcf.Reader(open(file))
    df = pd.DataFrame([vars(r) for r in reader])
    out = df.merge(pd.DataFrame(df.INFO.tolist()),
                   left_index=True, right_index=True)
    return out

In [12]:
read("./test_mutect.vcf")

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,start,end,alleles,samples,_sample_indexes,affected_start,affected_end,DB,SOMATIC,VT
0,1,2453125,None,C,[G],None,[REJECT],{},GT:AD:BQ:DP:FA,2453124,2453125,"[C, G]","[Call(sample=NORMAL, CallData(GT=0/1, AD=[160,...","{'NORMAL': 0, 'TUMOR': 1}",2453124,2453125,NaN,NaN,NaN
1,1,2587467,None,T,[A],None,[REJECT],{},GT:AD:BQ:DP:FA,2587466,2587467,"[T, A]","[Call(sample=NORMAL, CallData(GT=0, AD=[4, 0],...","{'NORMAL': 0, 'TUMOR': 1}",2587466,2587467,NaN,NaN,NaN
2,1,7400200,None,C,[G],None,[REJECT],{},GT:AD:BQ:DP:FA,7400199,7400200,"[C, G]","[Call(sample=NORMAL, CallData(GT=0, AD=[0, 0],...","{'NORMAL': 0, 'TUMOR': 1}",7400199,7400200,NaN,NaN,NaN
3,1,11399573,rs2993661,C,[T],None,[REJECT],{'DB': True},GT:AD:BQ:DP:FA,11399572,11399573,"[C, T]","[Call(sample=NORMAL, CallData(GT=0, AD=[2, 0],...","{'NORMAL': 0, 'TUMOR': 1}",11399572,11399573,True,NaN,NaN
4,1,12033120,rs3820190,G,[C],None,[REJECT],{'DB': True},GT:AD:BQ:DP:FA,12033119,12033120,"[G, C]","[Call(sample=NORMAL, CallData(GT=0/1, AD=[0, 7...","{'NORMAL': 0, 'TUMOR': 1}",12033119,12033120,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Y,13466317,None,A,[T],None,[REJECT],{},GT:AD:BQ:DP:FA,13466316,13466317,"[A, T]","[Call(sample=NORMAL, CallData(GT=0, AD=[3, 1],...","{'NORMAL': 0, 'TUMOR': 1}",13466316,13466317,NaN,NaN,NaN
496,Y,13475391,None,A,[T],None,[REJECT],{},GT:AD:BQ:DP:FA,13475390,13475391,"[A, T]","[Call(sample=NORMAL, CallData(GT=0, AD=[4, 0],...","{'NORMAL': 0, 'TUMOR': 1}",13475390,13475391,NaN,NaN,NaN
497,Y,13477095,None,T,[A],None,[REJECT],{},GT:AD:BQ:DP:FA,13477094,13477095,"[T, A]","[Call(sample=NORMAL, CallData(GT=0, AD=[37, 4]...","{'NORMAL': 0, 'TUMOR': 1}",13477094,13477095,NaN,NaN,NaN
498,Y,13486675,None,T,[C],None,[REJECT],{},GT:AD:BQ:DP:FA,13486674,13486675,"[T, C]","[Call(sample=NORMAL, CallData(GT=0, AD=[0, 6],...","{'NORMAL': 0, 'TUMOR': 1}",13486674,13486675,NaN,NaN,NaN
